In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame


In [ ]:
df = pd.read_csv(r"C:\Users\User\Desktop\Final_data_merged_v2.csv")

In [ ]:

df = pd.read_csv("your_input_file.csv")

df["Timestamp"] = pd.to_datetime(df["Timestamp"])

df = df.sort_values(["MMSI", "Timestamp"])

window_size = pd.Timedelta("1h")

#Generer et window_id for hver 1-times blok for hvert skib
df["window_id"] = (
    df.groupby("MMSI")["Timestamp"]
    .transform(lambda x: ((x - x.min()) // window_size).astype(int))
)

#Kombinér MMSI og vindue for unik ID
df["window_id"] = df["MMSI"].astype(str) + "_" + df["window_id"].astype(str)


value_columns = ["ROT", "SOG", "COG", "Heading"]

#Lav long-format dataframe til TSfresh
df_long = df[["window_id", "Timestamp"] + value_columns].melt(
    id_vars=["window_id", "Timestamp"],
    value_vars=value_columns,
    var_name="variable",
    value_name="value"
)

# 9. Omdøb så det matcher TSfresh's format
df_long = df_long.rename(columns={"window_id": "id", "Timestamp": "time"})

# 10. Kør TSfresh
features = extract_features(
    df_long,
    column_id="id",
    column_sort="time",
    column_kind="variable",
    column_value="value",
    disable_progressbar=True
)

#beregn også mean for hver feature i vindue
mean_features = df.groupby("window_id")[value_columns].mean().add_suffix("_mean")

#Træk MMSI og label ud fra første række i hvert vindue
meta = df.groupby("window_id").agg({
    "MMSI": "first",
    "Label": "first"
})

#Saml alt i ét slut-dataframe
final_df = pd.concat([features, mean_features, meta], axis=1).reset_index().rename(columns={"index": "window_id"})

#Gem som CSV
final_df.to_csv(r"C:\Users\User\Desktop\P2", index=False)
